# Introduction to Time Series Data

Data can come in many different formats, and many differentshapes and sizes. You've maybe heard of tabular data, a format you may be familiar with from working in something like Excel. 

We will explore two main kinds of tabular data in this module. The first is time series data. Time series data will be *indexed* with a date and time. We'll look a bit more closely at that soon, but for now just think of it as each row having a date or time, rather than a row number.

## Loading Data

One of the most popular packages in Python for working with tabular data is called Pandas. Today we'll get acquainted with Pandas.

The first thing we'll do is `import` the `pandas` package. Convention has us use a shortform name - `pd` - because we'll be using the package so often.

In [ ]:
import pandas as pd

This time, we're not using a pre-packaged dataset - we'll use pandas' `read_csv()` to load Apple trading data from a URL into a `DataFrame`.

In [ ]:
# Load the data
url = "https://raw.githubusercontent.com/ImperialCollegeLondon/efds-ta-python/main/data/AAPL_2020.csv"
df = pd.read_csv(url)

We can look at the DataFrame as we usually do.

In [ ]:
# Check the head
print(df.head())


In [ ]:
# Print summary info
print(df.info())


## Setting the Index

In a DataFrame, each row is assigned a unique index value. By default, this is just a number (starting at 0). With our Tips data, we only considered an index when plotting data that we had grouped and aggregated.

When it makes sense, we can choose one of the other columns to be an index. For time series data, where each row represents a different point in time, we'll set our `Date` column as the index. This will make it easier for us to work with the data, and can speed up other operations later on.

First, we have to convert our date column from a plain string to a datetime object.


In [ ]:
# Convert the 'Date' column to a datetime object
df['Date'] = pd.to_datetime(df['Date'])

# Set the 'Date' column as the index
df.set_index('Date', inplace=True)

We convert the 'Date' column to a datetime object because pandas can recognise and efficiently work with datetime objects. We set the `Date` column as the index because in time-series data like ours, operations are time-based.

With the index set, we can now use it to access different portions of our data a little bit more easily.

In [ ]:
# Access a row
print(df.loc['2020-08-18'])

# Access a specific cell
print(df.loc['2020-08-18', 'Close'])

# Access a range
print(df.loc["2020-08-18":"2020-08-20"])


### Exercise 1

Compare AAPL's *median* **high** in Q1 and Q2 of 2020. In which quarter was it higher? Use the cell below to show your work.

In [ ]:
## YOUR CODE GOES HERE

## Returns

Returns refer to the gain or loss made on an initial investment, often expressed as a percentage. We can use the generic **percentage change** formula here:

$$ (price_{end} - price_{start}) / price_{start} $$

We can apply this to close prices to calculate the simple daily return:

$$ (close price_{today} - close price_{yesterday}) / close price_{yesterday} $$

When available, use the Adjusted Close price, which takes into account corporate actions (dividends for example).

In [ ]:
aug18_closing = df.loc["2020-08-18", "Adj Close"]
aug17_closing = df.loc["2020-08-17", "Adj Close"]

aug18_return = (aug18_closing - aug17_closing) / aug17_closing
print(aug18_return)

This simple daily return expresses a loss in value of 2.5%

If we wanted to use the above approach to calculate daily returns for each day in our data set, it would take a long time. Let's see how we can use pandas `pct_change()` to make this sort of work easy, by applying our percentage change formula one column at a time.

In [ ]:
# Create a new column and populate it with daily returns
df['Daily Return'] = df['Adj Close'].pct_change()
df

Notice how the first row in our data has a missing value **NaN** in the new daily return column. This is because our data doesn't have a close price for the day before it!

What to do with this missing value depends on what further analysis we want to do. If we want to carry out simple descriptive statistics like compute the mean, max, or standard deviation,  we can leave our missing value as NaN, because pandas will by default ignore NaNs when calculating these.

In [ ]:
print("Average daily return:", df["Daily Return"].mean())

print("Highest daily return:", df["Daily Return"].max())

print("Standard Deviation:", df["Daily Return"].std())

For more complex analyses though, we may want to drop or fill this value. Let's calculate cumulative returns for the period. Instead of comparing a given day with the day before it, cumulative returns compare a given day with the first day of the period, to indicate how our stock has performed since our initial investment.

Because we're doing cumulative multiplication, we'll add 1 to the adjusted closing price, so we can compound the return over time.

In [ ]:
df["Cumulative Return"] = (1 + df["Daily Return"]).cumprod()

df

### Exercise 2

Calculate the daily change in trading volume for this stock, and then calculate the mean change in trading volume over the period. Take the same approach we used for calculating daily returns, but consider volume instead of the adjusted close price.

Then determine the trend in AAPL's stock this year:

- Rising volume and increasing price might indicate a **bullish** trend (where the uptrend is backed by strong demand and could continue).

- Rising volume and decreasing price might indicate a **bearish** trend (where the downtrend is backed by strong selling pressure and could continue).

- Falling volume on price increase or decrease often indicate a trend is losing strength. It might suggest that momentum is waning and a price reversal is coming.

In [ ]:
## YOUR CODE GOES HERE

## Surges

Surges in price or trading volume can be helpful indicators for traders. We usually define a surge as an increase on the day before by an amount higher than some defined threshold. A common threshold is two standard deviations above the mean. Let's look at price surges.

In [ ]:
# Find the mean return
mean_return = df["Daily Return"].mean()

# Define a threshold as two standard deviations above the mean
return_threshold = mean_return + (df["Daily Return"].std() * 2)

# Define a condition
condition = df["Daily Return"] > return_threshold

# Subset the dataframe where daily returns are higher than the threshold
df[condition]

## Moving Averages

Moving averages are a different kind of indicator, one that smooths out small variations in trading data to give a better picture of the overall trend.

A Simple Moving Average (SMA) is one which averages out a price over a specific period. The average is "moving" because when a new day is considered in the period, the oldest date is discarded.

Moving averages can be *fast*, when they cover a short period, or *slow* when they consider a longer period. The longer the period, the more those small variations are smoothed out.

In [ ]:
# Calculate a fast, 20-Day Moving Average
df['20-day MA'] = df['Adj Close'].rolling(window=20).mean()

# Calculate a slow, 200-Day Moving Average
df['200-day MA'] = df['Adj Close'].rolling(window=200).mean()

## Other useful functions

Another useful package in data science is NumPy. It actually powers alot of the work that pandas does, so technically, you've already used it!

One useful function is `np.where()`, often used for populating columns with a signal or indicator, depending on if a condition is met. Let's create a column to colour code our trading days. Days will have a different colour depening on if the market closes higher (green) or lower (red) than the opening.

In [ ]:
import numpy as np

df["colour"] = np.where(df["Close"] > df["Open"], "green", "red")
df